In [1]:
#! pip install --upgrade xarray zarr gcsfs cftime nc-time-axis

In [2]:
#! pip install xarray zarr gcsfs

In [3]:
#pip install dask

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs
import matplotlib.pyplot as plt
%matplotlib inline


xr.set_options(display_style='html')
%config InlineBackend.figure_format = 'retina' 

In [2]:
plt.rcParams['figure.figsize'] = 12, 6

In [3]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706


**

In [4]:
historical_precipitation = df[
   (df['institution_id'] == 'NCAR') &
   (df['source_id'] == 'CESM2') &
   (df['member_id'] == 'r11i1p1f1') &
   (df['activity_id'] == 'CMIP') &
   (df['experiment_id'] == 'historical') &
   (df['variable_id'].isin(['pr'])) &
   (df['table_id'].isin(['Amon'])) 
]
historical_precipitation

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
200657,CMIP,NCAR,CESM2,historical,r11i1p1f1,Amon,pr,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r1...,NaN,20190514


In [6]:
# Load historical precipitation data
gcs = gcsfs.GCSFileSystem(token='anon')

# Get the path
zstore = historical_precipitation.zstore.values[0]

# Open the dataset
ds_historical = xr.open_zarr(gcs.get_mapper(zstore), consolidated=True)
ds_historical

<xarray.Dataset> Size: 438MB
Dimensions:    (time: 1980, lat: 192, lon: 288, nbnd: 2)
Coordinates:
  * time       (time) object 16kB 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lat        (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 2kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
    lat_bnds   (lat, nbnd) float64 3kB ...
    lon_bnds   (lon, nbnd) float64 5kB ...
    time_bnds  (time, nbnd) object 32kB ...
Dimensions without coordinates: nbnd
Data variables:
    pr         (time, lat, lon) float32 438MB ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   674885.0
    branch_time_in_parent:  219000.0
    case_id:                972
    ...                     ...
    table_id:               Amon
    tracking_id:            hdl:21.14100/72897fd8-3516-431d-a1b1-c083130871a7...
    variable_id:            pr
    variant_info:           CMIP6 20th century experiments (1850-2014) with C...
    variant_label:          r11i1p1f1
    status:                 2019-10-25;created;by nhn2@columbia.edu

In [7]:
ds_historical['time'] = ds_historical.indexes['time'].to_datetimeindex()

C:\Users\zhy\AppData\Local\Temp\ipykernel_25856\1116224877.py:1: FutureWarning: In a future version of xarray to_datetimeindex will default to returning a 'us'-resolution DatetimeIndex instead of a 'ns'-resolution DatetimeIndex. This warning can be silenced by explicitly passing the `time_unit` keyword argument.
  ds_historical['time'] = ds_historical.indexes['time'].to_datetimeindex()
C:\Users\zhy\AppData\Local\Temp\ipykernel_25856\1116224877.py:1: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds_historical['time'] = ds_historical.indexes['time'].to_datetimeindex()


In [8]:
# Drop unnecessary coordinates and create 'year' coordinate
ds_historical = ds_historical.assign_coords(
    year = ds_historical.time.dt.year
)
ds_historical = ds_historical.drop_vars(['time', 'lat_bnds', 'lon_bnds', 'time_bnds'])
ds_historical

<xarray.Dataset> Size: 438MB
Dimensions:  (time: 1980, lat: 192, lon: 288)
Coordinates:
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    year     (time) int64 16kB 1850 1850 1850 1850 1850 ... 2014 2014 2014 2014
Dimensions without coordinates: time
Data variables:
    pr       (time, lat, lon) float32 438MB ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   674885.0
    branch_time_in_parent:  219000.0
    case_id:                972
    ...                     ...
    table_id:               Amon
    tracking_id:            hdl:21.14100/72897fd8-3516-431d-a1b1-c083130871a7...
    variable_id:            pr
    variant_info:           CMIP6 20th century experiments (1850-2014) with C...
    variant_label:          r11i1p1f1
    status:                 2019-10-25;created;by nhn2@columbia.edu

In [9]:
# Return the precipitation for each lon and lat grouped by year
his_annual_precip = ds_historical.groupby('year').mean(dim='time')
his_annual_precip

KeyboardInterrupt: 

In [12]:
#convert longitudes to -180 to 180 range
his_annual_precip = his_annual_precip.assign_coords(
    lon=(((his_annual_precip.lon + 180) % 360) - 180)
).sortby('lon')

**Compute the dataframe for each region's historical annual precipitation level**

In [13]:
northeast = his_annual_precip.sel(lat=slice(38,47), lon=slice(-80,-66))
south = his_annual_precip.sel(lat=slice(24,39), lon=slice(-106,-75))
midwest = his_annual_precip.sel(lat=slice(36,49), lon=slice(-104,-80))
west = his_annual_precip.sel(lat=slice(31,49), lon=slice(-125,-102))

In [14]:
northeast_america_precip = northeast.groupby('year').mean(dim=['lat', 'lon'])
northeast_hist_df = northeast_america_precip.to_dataframe().reset_index()
northeast_hist_df

,year,pr
0,1850,0.000034
1,1851,0.000043
2,1852,0.000040
3,1853,0.000038
4,1854,0.000041
...,...,...
160,2010,0.000046
161,2011,0.000040
162,2012,0.000043
163,2013,0.000044


In [15]:
south_america_precip = south.groupby('year').mean(dim=['lat', 'lon'])
south_hist_df = south_america_precip.to_dataframe().reset_index()
south_hist_df

,year,pr
0,1850,0.000029
1,1851,0.000030
2,1852,0.000033
3,1853,0.000030
4,1854,0.000036
...,...,...
160,2010,0.000031
161,2011,0.000037
162,2012,0.000035
163,2013,0.000033


In [16]:
midwest_america_precip = midwest.groupby('year').mean(dim=['lat', 'lon'])
midwest_hist_df = midwest_america_precip.to_dataframe().reset_index()
midwest_hist_df

,year,pr
0,1850,0.000020
1,1851,0.000021
2,1852,0.000023
3,1853,0.000025
4,1854,0.000026
...,...,...
160,2010,0.000023
161,2011,0.000023
162,2012,0.000026
163,2013,0.000025


In [17]:
west_america_precip = west.groupby('year').mean(dim=['lat', 'lon'])
northwest_hist_df = west_america_precip.to_dataframe().reset_index()
northwest_hist_df

,year,pr
0,1850,0.000017
1,1851,0.000019
2,1852,0.000019
3,1853,0.000022
4,1854,0.000019
...,...,...
160,2010,0.000018
161,2011,0.000016
162,2012,0.000019
163,2013,0.000018


In [18]:
# Under a file named 'historical_data'
northeast_hist_df.to_csv('historical_data/northeast_historical_precipitation.csv', index=False)
south_hist_df.to_csv('historical_data/south_historical_precipitation.csv', index=False)
midwest_hist_df.to_csv('historical_data/midwest_historical_precipitation.csv', index=False)
northwest_hist_df.to_csv('historical_data/northwest_historical_precipitation.csv', index=False)

**Future precipitation level prediction under low emission scenario(SSP126) and high emission scenario(SSP585)**

In [16]:
!pip install dask

In [5]:
import numpy as np
import pandas as pd
import xarray as xr
import gcsfs
import dask
import dask.array as da


future_low_day = df[
    (df['activity_id'] == 'ScenarioMIP') &
    (df['institution_id'] == 'NCAR') &
    (df['source_id'] == 'CESM2') &
    (df['member_id'] == 'r11i1p1f1') &
    (df['experiment_id'] == 'ssp126') &
    (df['variable_id'] == 'pr') &
    (df['table_id'] == 'day')
]

future_high_day = df[
    (df['activity_id'] == 'ScenarioMIP') &
    (df['institution_id'] == 'NCAR') &
    (df['source_id'] == 'CESM2') &
    (df['member_id'] == 'r11i1p1f1') &
    (df['experiment_id'] == 'ssp585') &
    (df['variable_id'] == 'pr') &
    (df['table_id'] == 'day')
]

In [6]:
import dask
import dask.array as da

gcs = gcsfs.GCSFileSystem(token='anon')

def open_and_preprocess_daily_pr(entry):
    
    zstore = entry.zstore.values[0]
    
    ds = xr.open_zarr(
        gcs.get_mapper(zstore),
        consolidated=True,
        chunks={'time': 365, 'lat': 40, 'lon': 40}
    )

    ds['time'] = ds.indexes['time'].to_datetimeindex()
    ds = ds.sel(time=ds.time.dt.year >= 2015)
    ds = ds.assign_coords(year=ds.time.dt.year)
    ds = ds.drop_vars(['lat_bnds', 'lon_bnds', 'time_bnds'], errors='ignore')
    ds = ds.assign_coords(
        lon=(((ds.lon + 180) % 360) - 180)
    ).sortby('lon')

    return ds

low_future_daily  = open_and_preprocess_daily_pr(future_low_day)   # SSP1-2.6
high_future_daily = open_and_preprocess_daily_pr(future_high_day)  # SSP5-8.5

C:\Users\zhy\AppData\Local\Temp\ipykernel_13132\1624486453.py:10: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 365. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_zarr(
C:\Users\zhy\AppData\Local\Temp\ipykernel_13132\1624486453.py:10: UserWarning: The specified chunks separate the stored chunks along dimension "lat" starting at index 40. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_zarr(
C:\Users\zhy\AppData\Local\Temp\ipykernel_13132\1624486453.py:10: UserWarning: The specified chunks separate the stored chunks along dimension "lon" starting at index 40. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_zarr(
C:\Users\zhy\AppData\Local\Temp\ipykernel_13132\1624486453.py:16: FutureWarning: In a future version of xarray to_datetimeindex will default to returning a 'us'-resolution DatetimeIndex instead 

In [7]:
HEAVY_THRESHOLD_MM = 10.0
SECONDS_PER_DAY = 86400.0


REGIONS = {
    "northeast": (slice(38, 47), slice(-80, -66)),
    "south":     (slice(24, 39), slice(-106, -75)),
    "midwest":   (slice(36, 49), slice(-104, -80)),
    "northwest": (slice(31, 49), slice(-125, -102)),
}

def compute_region_heavy_days(ds_daily, lat_slice, lon_slice,
                              threshold_mm=HEAVY_THRESHOLD_MM):
    
    # kg m-2 s-1 -> mm/day
    pr_mm_day = ds_daily['pr'] * SECONDS_PER_DAY

    
    region_pr = pr_mm_day.sel(lat=lat_slice, lon=lon_slice)

    
    region_daily_mean = region_pr.mean(dim=['lat', 'lon'])

    # heavy rain: ≥ 10mm/day
    heavy_bool = region_daily_mean >= threshold_mm

    
    heavy_days = heavy_bool.groupby('time.year').sum(dim='time')

   
    df_region = heavy_days.compute().to_dataframe(name='heavy_days').reset_index()
    df_region = df_region.rename(columns={'year': 'year'})
    return df_region

In [ ]:
# （SSP1-2.6）
low_heavy = {}
for name, (lat_s, lon_s) in REGIONS.items():
    low_heavy[name] = compute_region_heavy_days(
        low_future_daily, lat_s, lon_s
    )

# （SSP5-8.5）
high_heavy = {}
for name, (lat_s, lon_s) in REGIONS.items():
    high_heavy[name] = compute_region_heavy_days(
        high_future_daily, lat_s, lon_s
    )

In [ ]:
def combine_regions(heavy_dict):
    
    frames = []
    for region_name, df_region in heavy_dict.items():
        tmp = df_region.copy()
        tmp['region'] = region_name
        frames.append(tmp)
    all_df = pd.concat(frames, ignore_index=True)

    combined = (all_df
                .groupby('year')['heavy_days']
                .sum()               
                .reset_index()
                .rename(columns={'heavy_days': 'heavy_days_all_regions'}))
    return combined

low_all  = combine_regions(low_heavy)
high_all = combine_regions(high_heavy)


heavy_merged = low_all.merge(
    high_all,
    on='year',
    suffixes=('_ssp126', '_ssp585')
)

heavy_merged.head()

In [ ]:
heavy_merged.to_csv(
    "us_heavy_rain_days_10mm.csv",
    index=False
)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

df = heavy_merged

plt.figure(figsize=(10,6))
plt.plot(df["year"], df["heavy_days_all_regions_ssp126"], label="SSP1-2.6")
plt.plot(df["year"], df["heavy_days_all_regions_ssp585"], label="SSP5-8.5")
plt.xlabel("Year")
plt.ylabel("Number of heavy rain days")
plt.title("Extreme Events: Heavy Rain Days per Year")
plt.legend()
plt.tight_layout()
plt.show()